<a href="https://colab.research.google.com/github/AlirezaGhavidel70/training-and-testing-deploying-the-AI-decision-agents/blob/main/Train_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @ Authors: Ao Du and Alireza Ghavidel
# This code trains an AI agent based on the user inputs
###############################################################################
################# User input values for trainig the AI-agent ##################
###############################################################################

# Specify the life horizon of the bridge
life_span = 30
# contribution weights of indirect cost
w1=0.05
# Please specify the city: Mamphis (0) or San Francisco =1
City = 0 # seismic region: 0: Memphis, 1: San Francisco, 2: other cities (you need to define the corresponding hazard curve)
# consider retrofiting
steel_lacketing = False # if there is steel jacketing retrofiting use "True" otherwise it should be "False"
seat_extender = False # if there is seat extender retrofiting use "True" otherwise it should be "False"
shear_key = False # if there is shear key retrofiting use "True" otherwise it should be "False"

# Specify the number of episodes to train the agent
num_episodes = int(40000)


## Hyperparameters and Utilities
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# LR is the learning rate of the ``Adam`` optimizer
BATCH_SIZE = 32
GAMMA = 0.96
EPS_START = 1.0
EPS_END = 0.01
EPS_DECAY = 5000
LR = 1e-2

###############################################################################
###############################################################################
###############################################################################

In [ ]:
# Code starts from here*-
"""
Fragility curves and risk analysis are incorporated in this code to find the earthquake damage state probabilities

@author: Alireza Ghavidel
"""
import numpy as np
import scipy
import sympy
from scipy import special
from sympy import *
import matplotlib.pyplot as plt
from scipy.misc import derivative
from scipy import *
from scipy.stats import lognorm
from scipy.optimize import minimize, rosen, rosen_der
from scipy import special
from scipy.stats import norm

class FRAG_DRL:

    def __init__(self, time_t=[], jacket_time=[], extender_time=[], shearKey_time=[], RC_time=[], EB_time=[], city_hazard = 0):


        self.time_t=np.array(time_t)
        self.extender_time=np.array(extender_time)
        self.jacket_time= np.array(jacket_time)
        self.shearKey_time= np.array(shearKey_time)
        self.RC_time= np.array(RC_time)
        self.EB_time= np.array(EB_time)
        self.city_hazard = city_hazard


    def mod_frag(pf,x,mod_factors):

        f = lambda x,mu,sigma: scipy.stats.lognorm.cdf(x, sigma, loc=0, scale=np.exp(mu))
        x0 = np.asarray([np.log(1.5), 0.001])
        mu,sigma = scipy.optimize.curve_fit(f,x,pf,p0=x0,method='lm',maxfev=5000)[0]
        mu=np.log(np.exp(mu)*mod_factors[0])
        sigma=sigma*mod_factors[1]
        pf_mod = scipy.stats.lognorm.cdf(x, sigma, loc=0, scale=np.exp(mu))
        return pf_mod, mu,sigma


    def fraglity (self,):
        # To find the constant values, please refer to the paper "Padgett J E, Dennemann K and Ghosh J 2010 Risk-based seismic life-cycle cost–benefit
        #(LCC-B) analysis for bridge retrofit assessment Structural Safety 32 165–73 "
        frag_mod_factors_SJ= [1.03,1,1.16,1,1.17,1,1.20,1]
        frag_mod_factors_EB= [2.94,1,1.31,1,1.21,1,1.17,1]
        frag_mod_factors_RC= [1.04,1,0.96,1,1.01,1,1.05,1]
        frag_mod_factors_SE= [1.01,1,1.00,1,1.00,1,1.31,1]
        frag_mod_factors_SK= [1.01,1,0.98,1,0.99,1,1.01,1]
        frag_mod_factors_RC_SK= [1.04,1,0.96,1,1.04,1,1.12,1]
        frag_mod_factors_SE_SK= [1.01,1,0.97,1,0.99,1,1.37,1]
        frag_mod_factors_SJ_SE= [1.03,1,1.16,1,1.17,1,1.31,1]
        frag_mod_factors_SJ_SK= [1.03,1,1.16,1,1.17,1,1.37,1]
        frag_mod_factors_SJ_SE_SK=[1.03,1,1.16,1,1.17,1,1.37,1]



        PGA=np.linspace(np.log(0.01), np.log(3), 30)
        PGA=np.exp(PGA)
        size=PGA.size



        lam_factor_complete_sys= np.zeros(size)
        lam_factor_extensive_sys= np.zeros(size)
        lam_factor_moderate_sys= np.zeros(size)
        lam_factor_slight_sys= np.zeros(size)



        # To find the constant values, please refer to the paper "Padgett J E, Dennemann K and Ghosh J 2010 Risk-based seismic life-cycle cost–benefit
        #(LCC-B) analysis for bridge retrofit assessment Structural Safety 32 165–73 "
        P_slight_sys = norm.cdf((np.log(PGA)-np.log(0.21))/0.69)
        P_moderate_sys = norm.cdf((np.log(PGA)-np.log(0.61))/0.60)
        P_extensive_sys = norm.cdf((np.log(PGA)-np.log(0.86))/0.60)
        P_complete_sys = norm.cdf((np.log(PGA)-np.log(1.20))/0.61)





        x= Symbol('x')

        if self.city_hazard == 0:
            y2= 11.8246*sympy.exp(56.1606*((sympy.log(x/134.7200))**-1)) # it is for Memphis
        elif self.city_hazard == 1:
            y2= 66.1763*sympy.exp(48.3592*((sympy.log(x/47.2546))**-1)) # it is for San Fransisco
        else:
            print('If you want to analysis based on your desired city, \
            refer to USGS website (https://earthquake.usgs.gov/hazards/interactive/) and\
            paper entitled "Improved seismic hazard model with application to probabilistic seismic demand analysis"\
            to add the corresponding hazard curve')

        y2prime=y2.diff(x)
        yprime=y2prime
        f = lambdify(x, yprime, 'numpy')

        s=np.zeros(size)
        s1 = np.zeros(size)



        lam_factor_complete_sys= np.zeros(size)
        lam_factor_extensive_sys= np.zeros(size)
        lam_factor_moderate_sys= np.zeros(size)
        lam_factor_slight_sys= np.zeros(size)



        for sa in range (0,size):
            s[sa]=np.abs(f(PGA[sa]))
            if sa!=0:
                s1[sa]=s[sa]*np.abs((PGA[sa]-PGA[sa-1]))


        if self.time_t>=self.extender_time and self.time_t>=self.jacket_time and self.time_t>=self.shearKey_time:


            P_slight_sys = norm.cdf((np.log(PGA)-np.log(0.21*frag_mod_factors_SJ_SE_SK[0]))/(0.69*frag_mod_factors_SJ_SE_SK[1]))
            P_moderate_sys = norm.cdf((np.log(PGA)-np.log(0.61*frag_mod_factors_SJ_SE_SK[2]))/(0.60*frag_mod_factors_SJ_SE_SK[3]))
            P_extensive_sys = norm.cdf((np.log(PGA)-np.log(0.86*frag_mod_factors_SJ_SE_SK[4]))/(0.60*frag_mod_factors_SJ_SE_SK[5]))
            P_complete_sys = norm.cdf((np.log(PGA)-np.log(1.20*frag_mod_factors_SJ_SE_SK[6]))/(0.61*frag_mod_factors_SJ_SE_SK[7]))



            lam_factor_complete_sys= s1*P_complete_sys
            lam_factor_extensive_sys= s1*P_extensive_sys
            lam_factor_moderate_sys= s1*P_moderate_sys
            lam_factor_slight_sys= s1*P_slight_sys

        elif self.time_t>=self.extender_time and self.time_t>=self.jacket_time:


            P_slight_sys = norm.cdf((np.log(PGA)-np.log(0.21*frag_mod_factors_SJ_SE[0]))/(0.69*frag_mod_factors_SJ_SE[1]))
            P_moderate_sys = norm.cdf((np.log(PGA)-np.log(0.61*frag_mod_factors_SJ_SE[2]))/(0.60*frag_mod_factors_SJ_SE[3]))
            P_extensive_sys = norm.cdf((np.log(PGA)-np.log(0.86*frag_mod_factors_SJ_SE[4]))/(0.60*frag_mod_factors_SJ_SE[5]))
            P_complete_sys = norm.cdf((np.log(PGA)-np.log(1.20*frag_mod_factors_SJ_SE[6]))/(0.61*frag_mod_factors_SJ_SE[7]))



            lam_factor_complete_sys= s1*P_complete_sys
            lam_factor_extensive_sys= s1*P_extensive_sys
            lam_factor_moderate_sys= s1*P_moderate_sys
            lam_factor_slight_sys= s1*P_slight_sys

        elif self.time_t>=self.shearKey_time and self.time_t>=self.jacket_time:


            P_slight_sys = norm.cdf((np.log(PGA)-np.log(0.21*frag_mod_factors_SJ_SK[0]))/(0.69*frag_mod_factors_SJ_SK[1]))
            P_moderate_sys = norm.cdf((np.log(PGA)-np.log(0.61*frag_mod_factors_SJ_SK[2]))/(0.60*frag_mod_factors_SJ_SK[3]))
            P_extensive_sys = norm.cdf((np.log(PGA)-np.log(0.86*frag_mod_factors_SJ_SK[4]))/(0.60*frag_mod_factors_SJ_SK[5]))
            P_complete_sys = norm.cdf((np.log(PGA)-np.log(1.20*frag_mod_factors_SJ_SK[6]))/(0.61*frag_mod_factors_SJ_SK[7]))



            lam_factor_complete_sys= s1*P_complete_sys
            lam_factor_extensive_sys= s1*P_extensive_sys
            lam_factor_moderate_sys= s1*P_moderate_sys
            lam_factor_slight_sys= s1*P_slight_sys

        elif self.time_t>=self.extender_time and self.time_t>=self.shearKey_time:


            P_slight_sys = norm.cdf((np.log(PGA)-np.log(0.21*frag_mod_factors_SE_SK[0]))/(0.69*frag_mod_factors_SE_SK[1]))
            P_moderate_sys = norm.cdf((np.log(PGA)-np.log(0.61*frag_mod_factors_SE_SK[2]))/(0.60*frag_mod_factors_SE_SK[3]))
            P_extensive_sys = norm.cdf((np.log(PGA)-np.log(0.86*frag_mod_factors_SE_SK[4]))/(0.60*frag_mod_factors_SE_SK[5]))
            P_complete_sys = norm.cdf((np.log(PGA)-np.log(1.20*frag_mod_factors_SE_SK[6]))/(0.61*frag_mod_factors_SE_SK[7]))



            lam_factor_complete_sys= s1*P_complete_sys
            lam_factor_extensive_sys= s1*P_extensive_sys
            lam_factor_moderate_sys= s1*P_moderate_sys
            lam_factor_slight_sys= s1*P_slight_sys

        elif self.time_t>=self.RC_time and self.time_t>=self.shearKey_time:


            P_slight_sys = norm.cdf((np.log(PGA)-np.log(0.21*frag_mod_factors_RC_SK[0]))/(0.69*frag_mod_factors_RC_SK[1]))
            P_moderate_sys = norm.cdf((np.log(PGA)-np.log(0.61*frag_mod_factors_RC_SK[2]))/(0.60*frag_mod_factors_RC_SK[3]))
            P_extensive_sys = norm.cdf((np.log(PGA)-np.log(0.86*frag_mod_factors_RC_SK[4]))/(0.60*frag_mod_factors_RC_SK[5]))
            P_complete_sys = norm.cdf((np.log(PGA)-np.log(1.20*frag_mod_factors_RC_SK[6]))/(0.61*frag_mod_factors_RC_SK[7]))



            lam_factor_complete_sys= s1*P_complete_sys
            lam_factor_extensive_sys= s1*P_extensive_sys
            lam_factor_moderate_sys= s1*P_moderate_sys
            lam_factor_slight_sys= s1*P_slight_sys

        elif self.time_t>=self.shearKey_time:


            P_slight_sys = norm.cdf((np.log(PGA)-np.log(0.21*frag_mod_factors_SK[0]))/(0.69*frag_mod_factors_SK[1]))
            P_moderate_sys = norm.cdf((np.log(PGA)-np.log(0.61*frag_mod_factors_SK[2]))/(0.60*frag_mod_factors_SK[3]))
            P_extensive_sys = norm.cdf((np.log(PGA)-np.log(0.86*frag_mod_factors_SK[4]))/(0.60*frag_mod_factors_SK[5]))
            P_complete_sys = norm.cdf((np.log(PGA)-np.log(1.20*frag_mod_factors_SK[6]))/(0.61*frag_mod_factors_SK[7]))



            lam_factor_complete_sys= s1*P_complete_sys
            lam_factor_extensive_sys= s1*P_extensive_sys
            lam_factor_moderate_sys= s1*P_moderate_sys
            lam_factor_slight_sys= s1*P_slight_sys


        elif self.time_t>=self.extender_time:


            P_slight_sys = norm.cdf((np.log(PGA)-np.log(0.21*frag_mod_factors_SE[0]))/(0.69*frag_mod_factors_SE[1]))
            P_moderate_sys = norm.cdf((np.log(PGA)-np.log(0.61*frag_mod_factors_SE[2]))/(0.60*frag_mod_factors_SE[3]))
            P_extensive_sys = norm.cdf((np.log(PGA)-np.log(0.86*frag_mod_factors_SE[4]))/(0.60*frag_mod_factors_SE[5]))
            P_complete_sys = norm.cdf((np.log(PGA)-np.log(1.20*frag_mod_factors_SE[6]))/(0.61*frag_mod_factors_SE[7]))



            lam_factor_complete_sys= s1*P_complete_sys
            lam_factor_extensive_sys= s1*P_extensive_sys
            lam_factor_moderate_sys= s1*P_moderate_sys
            lam_factor_slight_sys= s1*P_slight_sys

        elif self.time_t>=self.RC_time:


            P_slight_sys = norm.cdf((np.log(PGA)-np.log(0.21*frag_mod_factors_RC[0]))/(0.69*frag_mod_factors_RC[1]))
            P_moderate_sys = norm.cdf((np.log(PGA)-np.log(0.61*frag_mod_factors_RC[2]))/(0.60*frag_mod_factors_RC[3]))
            P_extensive_sys = norm.cdf((np.log(PGA)-np.log(0.86*frag_mod_factors_RC[4]))/(0.60*frag_mod_factors_RC[5]))
            P_complete_sys = norm.cdf((np.log(PGA)-np.log(1.20*frag_mod_factors_RC[6]))/(0.61*frag_mod_factors_RC[7]))



            lam_factor_complete_sys= s1*P_complete_sys
            lam_factor_extensive_sys= s1*P_extensive_sys
            lam_factor_moderate_sys= s1*P_moderate_sys
            lam_factor_slight_sys= s1*P_slight_sys


        elif self.time_t>=self.EB_time:


            P_slight_sys = norm.cdf((np.log(PGA)-np.log(0.21*frag_mod_factors_EB[0]))/(0.69*frag_mod_factors_EB[1]))
            P_moderate_sys = norm.cdf((np.log(PGA)-np.log(0.61*frag_mod_factors_EB[2]))/(0.60*frag_mod_factors_EB[3]))
            P_extensive_sys = norm.cdf((np.log(PGA)-np.log(0.86*frag_mod_factors_EB[4]))/(0.60*frag_mod_factors_EB[5]))
            P_complete_sys = norm.cdf((np.log(PGA)-np.log(1.20*frag_mod_factors_EB[6]))/(0.61*frag_mod_factors_EB[7]))



            lam_factor_complete_sys= s1*P_complete_sys
            lam_factor_extensive_sys= s1*P_extensive_sys
            lam_factor_moderate_sys= s1*P_moderate_sys
            lam_factor_slight_sys= s1*P_slight_sys

        elif self.time_t>=self.jacket_time:


            P_slight_sys = norm.cdf((np.log(PGA)-np.log(0.21*frag_mod_factors_SJ[0]))/(0.69*frag_mod_factors_SJ[1]))
            P_moderate_sys = norm.cdf((np.log(PGA)-np.log(0.61*frag_mod_factors_SJ[2]))/(0.60*frag_mod_factors_SJ[3]))
            P_extensive_sys = norm.cdf((np.log(PGA)-np.log(0.86*frag_mod_factors_SJ[4]))/(0.60*frag_mod_factors_SJ[5]))
            P_complete_sys = norm.cdf((np.log(PGA)-np.log(1.20*frag_mod_factors_SJ[6]))/(0.61*frag_mod_factors_SJ[7]))



            lam_factor_complete_sys= s1*P_complete_sys
            lam_factor_extensive_sys= s1*P_extensive_sys
            lam_factor_moderate_sys= s1*P_moderate_sys
            lam_factor_slight_sys= s1*P_slight_sys



        else:


            lam_factor_complete_sys= s1*P_complete_sys
            lam_factor_extensive_sys= s1*P_extensive_sys
            lam_factor_moderate_sys= s1*P_moderate_sys
            lam_factor_slight_sys= s1*P_slight_sys




        annual_rate_slight_sys=sum(lam_factor_slight_sys)
        annual_rate_moderate_sys=sum(lam_factor_moderate_sys)
        annual_rate_extensive_sys=sum(lam_factor_extensive_sys)
        annual_rate_complete_sys=sum(lam_factor_complete_sys)





        PT_slight_sys=1-np.exp(-annual_rate_slight_sys)
        PT_moderate_sys=1-np.exp(-annual_rate_moderate_sys)
        PT_extensive_sys=1-np.exp(-annual_rate_extensive_sys)
        PT_complete_sys=1-np.exp(-annual_rate_complete_sys)



        PP_sys=[1-(PT_slight_sys), -PT_moderate_sys+PT_slight_sys, -PT_extensive_sys+PT_moderate_sys,-PT_complete_sys+PT_extensive_sys, PT_complete_sys]

        P_sys = PP_sys

        return P_sys



In [ ]:
# -*- coding: utf-8 -*-
"""
Bridge Environment is developed here
@author: Alireza Ghavidel
"""

from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
#from Fragility_risk_analysis import FRAG_DRL
from copy import deepcopy

class BridgeEnvFrg(Env):
    def __init__(self, life, city):

        # lets consider the order as following for components:
        # "0": Deck, "1":Super structure, "2": Sub structure,
        # Actions: we can take : Do nothing, Minor, Major and replacement
        self.action_space = Discrete(64) # 64: 4*4*4 actions //// 4 actions [do nothing, minor, major, replacement] for deck, superstructure and substructure
        # observation array
        self.observation_space = Box(low=np.array([0, 0, 0]), high=np.array([8,8,8]), shape = (3,), dtype=np.int64) # [Deck 0-8, Sperstructure 0-8, Substructure 0-8]
        # Set start state
        self.state = np.zeros(shape=(3,),dtype = np.int64)
        self.state[0:3] =  np.random.randint(4,[9,9,9])    # initiating the initial state from the fair condition randomly
        self.total_steps = 0 # counting
        self.componenets = 3 # number of components
        self.life = life # life horizon
        self.city = city # city for risk analysis: 0: Memphis, 1: San Francisco
        self.damage = 0


    def step(self, action, Deck_area, L_d, ADT, r_Truck, jacketing = False, jacketing_t = 100, extender = False, extender_t = 100, shearKey = False, shearKey_t = 100, w1=0.05):
        # Markov transition matrices can be modified by the user if needed
        # condition-based rating
        state_old = deepcopy(self.state)
        trans = np.zeros(shape=[self.componenets,4,9,9])  # 3 comps, 4 actions, 9 states
        # Do nothing
        trans [0, 0] = [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0114, 0.9886, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0174, 0.9826, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0000, 0.0227, 0.9773, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0268, 0.9732, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0714, 0.9286, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0452, 0.1011, 0.8537]]

        trans [1, 0] = [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0282, 0.9718, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0107, 0.9893, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0147, 0.9853, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0194, 0.9806, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0000, 0.0246, 0.9754, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0276, 0.9724, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0532, 0.9468, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0282, 0.0995, 0.8723]]

        trans [2, 0] = [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0142, 0.9858, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0208, 0.9792, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0302, 0.9698, 0.0000, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0000, 0.0282, 0.9718, 0.0000, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0207, 0.9793, 0.0000, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0654, 0.9346, 0.0000],
                        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0349, 0.0925, 0.8726]]


        # Minor maintenance
        trans [0:3, 1] = [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                          [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                          [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                          [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                          [0.0, 0.0, 0.0, 0.0, 0.9, 0.1, 0.0, 0.0, 0.0],
                          [0.0, 0.0, 0.0, 0.0, 0.0, 0.80, 0.20, 0.0, 0],
                          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, 0.0],
                          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6, 0.4],
                          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]

        # Major Maintenance
        trans [0:3, 2] = [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                          [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                          [0.0, 0.0, 0.3, 0.4, 0.3, 0.0, 0.0, 0.0, 0.0],
                          [0.0, 0.0, 0.0, 0.1, 0.3, 0.6, 0.0, 0.0, 0.0],
                          [0.0, 0.0, 0.0, 0.0, 0.1, 0.2, 0.7, 0.0, 0.0],
                          [0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.1, 0.85, 0],
                          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.9],
                          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0.02, 0.98],
                          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]



        # Replace
        trans [0:3,3,:,8] = 1

        # unravel actions
        action_index = np.unravel_index(action,(4,4,4))  # (deck, superstructure, substructure, column, bearing, abutment, retrofitting combinations )
        action_index = np.asarray(action_index)

        # Damage State (DS) only reflect the DS happened in each year based on the seismic risk model. We don't propagate the DS to the next year
        self.damage = 0


        # Retrofitting configuration: we have 3 possible retrofitting actions: Seat extender, steel jacketing, and shear key
        if extender == True:
            self.extender_time = extender_t
        else:
            self.extender_time = []
        if jacketing == True:
            self.jacket_time = jacketing_t
        else:
            self.jacket_time = []
        if shearKey == True:
            self.shearKey_time = shearKey_t
        else:
            self.shearKey_time = []


        # here we call the fragility and risk analysis part to get the damage state probabilities
        FRG=FRAG_DRL(time_t=self.total_steps, jacket_time=self.jacket_time, extender_time=self.extender_time, shearKey_time=self.shearKey_time, RC_time=[], EB_time=[], city_hazard = self.city)
        P_sys=FRG.fraglity()



        # this part the next state is calculated based on the transition probabilities come from Markov matrices and fragility functions
        for component in range(self.componenets):
            self.state[component] = np.random.choice(9, p=trans[component][action_index[component]][self.state[component]])

        # Earthquake damage type
        self.damage = np.random.choice(5, p=P_sys)



        # Mapping DS to CR
        state_update = deepcopy(self.state)

        if self.damage == 0:
            self.state[0] = state_update[0]
            self.state[1] = state_update[1]
            self.state[2] = state_update[2]

        elif self.damage == 1:
            self.state[0] = np.min([state_update[0],6])
            self.state[1] = np.min([state_update[1],6])
            self.state[2] = np.min([state_update[2],6])

        elif self.damage == 2:
            self.state[0] = np.min([state_update[0],4])
            self.state[1] = np.min([state_update[1],4])
            self.state[2] = np.min([state_update[2],4])

        elif self.damage == 3:
            self.state[0] = np.min([state_update[0],3])
            self.state[1] = np.min([state_update[1],3])
            self.state[2] = np.min([state_update[2],3])

        elif self.damage == 4:
            self.state[0] = 0
            self.state[1] = 0
            self.state[2] = 0


        # update CS based on actions

        if action_index[1]==3:
            self.state[0]=8
            self.state[1]=8
        if action_index[2]==3:
            self.state[0] = 8
            self.state[1] = 8
            self.state[2] = 8



        """Calculate reward'"""
        # Costant values for reward calculations can be adjusted by the user, if needed
        # Residual serviceability based on current action
        if max(action_index[0:3]) ==0: # Do nothing
            T1 = 0.0
            alpha1 = 1.0
        elif max(action_index[0:3])==1: # Minor maintenance
            T1 = 7.0
            alpha1 = 0.75
        elif max(action_index[0:3])==2: # Major maintenance
            T1 = 30.0
            alpha1 = 0.5
        else:                           # replace
            T1 = 182.0
            alpha1 = 0.0

        T_CR = T1
        alpha_CR = alpha1

        # final T and alpha from different maintenance actions and different damage states
        T = T_CR
        alpha = alpha_CR


        # Bridge total construction costs
        C_b = 160.19 # Bridge construction cost rate $/ft2 deck area for Tennessee
        C_Bridge = Deck_area*C_b * 10.764  # 10.764 convert from meter square to feet square

        # Retrofitting costs (Huang Y, Parmelee S and Pang W 2014 Optimal retrofit scheme for highway network under seismic hazards
        #International Journal of Transportation Science and Technology 3 109–28)
        steel_jacketing_unit = 0.12  #
        C_jacketing = steel_jacketing_unit *  C_Bridge
        C_seat_extender= 0.03 * C_Bridge
        C_shear_key= 0.03 * C_Bridge
        C_bearing = 0.05 * C_Bridge
        """ Direct agency costs """
        # refer to "Elbehairy H 2007 Bridge management system with integrated life cycle cost optimization"
        Cost_matrix = np.zeros(shape=(3,4))
        Cost_matrix[0:3,0] = 0.0                                    # Components: Deck, superstructure, substructure / action: do nothing
        Cost_matrix[0,1] = 0.05*C_Bridge * 0.225                    # Components: Deck / action: minor
        Cost_matrix[1,1] = 0.05*C_Bridge * 0.263                    # Components: superstructure  / action: minor
        Cost_matrix[2,1] = 0.05*C_Bridge * 0.412                    # Components: substructure / action: minor
        Cost_matrix[0,2] = 0.25*C_Bridge * 0.225                    # Components: Deck / action: major
        Cost_matrix[1,2] = 0.25*C_Bridge * 0.263                    # Components: superstructure / action: major
        Cost_matrix[2,2] = 0.25*C_Bridge * 0.412                    # Components: substructure / action: major
        Cost_matrix[0,3] = 1.1*C_Bridge * 0.225                     # Components: Deck / action: rebuild
        Cost_matrix[1,3] = 1.1*C_Bridge * (0.263 + 0.225)           # Components: superstructure / action: rebuild
        Cost_matrix[2,3] = 1.1*C_Bridge * (0.412 + 0.263 + 0.225)   # Components: substructure / action: rebuild


        C_CR = 0.0
        C_Ret = 0.0
        for i in range(0,3):  # Summation over all the 3 components
            C_CR += Cost_matrix[i,int(action_index[i])]

        C_j = 0.0
        C_ex= 0.0
        C_sk = 0.0
        if jacketing == True and self.total_steps==np.asarray(self.jacket_time):
            C_j = C_jacketing
        else:
            C_j=0.0

        if extender == True and self.total_steps==np.asarray(self.extender_time):
            C_ex = C_seat_extender
        else:
            C_ex = 0.0

        if shearKey == True and self.total_steps==np.asarray(self.shearKey_time):
            C_sk = C_shear_key
        else:
            C_sk = 0.0

        C_Ret = C_j + C_ex + C_sk
        C_Direct = 0
        C_Direct = C_CR + C_Ret


        if C_Direct > 1.1*C_Bridge:   # Avoid double counting the reconstruction action
            C_Direct = 1.1*C_Bridge
        """ Indirect social costs """ # the constat values can be changed by the user
        L_d = L_d * 0.621371     # 0.621371 converts km to mile
        C_O_Car = 0.64 # Running cost for car ($/car/mile)
        C_O_Truck = 1.855 # Running cost for truck ($/truck/mile)
        C_W_Car = 12.35 # Time costs for car passengers ($/hr/passenger)
        C_W_Truck = 31.05 # Time costs for trucks ($/hr/truck)
        O_Car = 1.67 # Number of passengers per car
        V_d = 50 # Detour speed mph

        # Determine percentage of detoured traffic due to serviceability loss (before the action)

        if state_old[0] >= 6: # deck
            phi_deck_old = 1
        elif state_old[0] >= 4 and state_old[0] < 6: # Deck
            phi_deck_old = 1 - (((6-state_old[0])/6)*0.75)
        else:
            phi_deck_old = 0.0

        if state_old[1] >= 4: # Superstructure
            phi_girder_old = 1.0
        else:
            phi_girder_old = 0.0

        if state_old[2] >= 4: # Substructure
            phi_sub_old = 1.0
        else:
            phi_sub_old = 0.0

        phi_old1 = min(phi_deck_old,phi_girder_old,phi_sub_old)


        phi_old = phi_old1

        # Indirect operation costs for all

        C_O = (C_O_Car*(1-r_Truck) + C_O_Truck*r_Truck)*L_d*ADT*((1-alpha*phi_old)*T + (1-phi_old)*(365-T))
        # Indirect time costs
        C_T = (C_W_Car*O_Car*(1-r_Truck) + C_W_Truck*r_Truck)*ADT*L_d/V_d*((1-alpha*phi_old)*T + (1-phi_old)*(365-T))
        # Total indirect costs
        C_Indirect = C_O + C_T

        # Calculate reward
        C_Indirect = 1.43 * C_Indirect # 1.43 is a factor to consider safety cost (accidents)
        reward = -(C_Direct + (w1 * C_Indirect)) # Negative of the total cost as the reward, w1 is the agency weighting factor

         # go to the next year
        self.total_steps +=1

        # Check it is done
        if self.total_steps  == self.life:
            done = True
        else:
            done = False

        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, C_Direct, C_Indirect,reward, done, info,self.damage


    def render(self):
        # Implement viz
        pass

    def reset(self):
        self.state = np.zeros(shape=(3,),dtype = np.int64)
        self.state[0:3] = np.random.randint(4,[9,9,9])
        self.total_steps = 0
        self.damage = 0

        return self.state




In [ ]:
# -*- coding: utf-8 -*-
"""
Parameterized DQN with Prioritized Experience Replay Memory (PER)
@authors: Dr. Ao Du and Alireza Ghavidel

"""
#!pip install gym
#import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time


time_jacketing= 0 # if there is steel jaketing retrofitting, please specify what year happened (when steel jacekting is "True")
time_extender= 0 # when seat_extender = True, please specify the year
time_shearKey= 0 # when seat_extender = True, please specify the year
#from Bridge_Environment import BridgeEnvFrg


start_time = time.time()


env = BridgeEnvFrg(life = life_span, city = City)
state_shape = env.observation_space.shape
n_params = int(4)
# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state = env.reset()
n_observations = len(state)


# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device', device)
## Replay Memory
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'A_deck','L_D','ADT','r_truck'))


class PrioritizedReplayMemory(object):

    def __init__(self, capacity, alpha=0.6, beta=0.4, beta_annealing=0.001):
        #alpha, beta, beta_annealing: Parameters used in prioritized experience replay.
        #alpha determines how much priority influences the sampling probability,
        #while beta and beta_annealing are used to control importance weights
        self.capacity = capacity
        self.alpha = alpha
        self.beta = beta
        self.beta_annealing = beta_annealing
        self.memory = deque([], maxlen=capacity)
        self.priorities = deque([], maxlen=capacity)
        self.position = 0
        self.size = 0  # Initialize size counter

    def push(self, *args):
        """Save a transition"""
        transition = Transition(*args)
        max_priority = max(self.priorities) if self.priorities else 1.0
        if self.size < self.capacity:  # Check if not at max capacity
            self.size += 1
        else:
            self.memory.popleft()  # Remove oldest transition
            self.priorities.popleft()  # Remove oldest priority
        self.memory.append(transition)
        self.priorities.append(max_priority)

    def sample(self, batch_size):
        prios = np.array(self.priorities)
        probs = prios ** self.alpha
        probs /= probs.sum()

        indices = np.random.choice(len(self.memory), batch_size, p=probs)
        samples = [self.memory[idx] for idx in indices]

        beta = self.beta + self.beta_annealing
        importance_weights = (len(self.memory) * probs[indices]) ** (-beta)
        importance_weights /= importance_weights.max()

        return samples, indices, importance_weights

    def update_priorities(self, indices, priorities):
        for idx, priority in zip(indices, priorities):
            self.priorities[idx] = priority

    def __len__(self):
        return self.size


# DQN Agent
class DQN(nn.Module):

    def __init__(self, n_inputs, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_inputs, 24)
        self.layer2 = nn.Linear(24, 24)
        self.layer3 = nn.Linear(24, n_actions)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

def select_action(inputs):
    global i_episode, previous_actions_deck, previous_actions_super, previous_actions_sub, test_on
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * i_episode / EPS_DECAY)

    if sample > eps_threshold:
        with torch.no_grad():
            inputs_cpu = inputs.to('cpu').numpy()
            action_values = policy_net(inputs)
            max_action = action_values.max(1)[1]  # Get the index of the maximum action

            # Convert max_action to a numpy array and then unravel it
            max_action_numpy = max_action.cpu().detach().numpy()
            modified_action = np.asarray(np.unravel_index(max_action_numpy, (4, 4, 4)))

            if test_on == 1:
                if inputs_cpu[0][0] >= 6 :
                    modified_action[0] = 0

                if inputs_cpu[0][1] >= 6 :
                    modified_action[1] = 0

                if inputs_cpu[0][2] >= 6 :
                    modified_action[2] = 0
            # Constraint: Check the previous actions for each component

            if any(action != 0 for action in  previous_actions_deck) and inputs_cpu[0][0] >= 4:
                modified_action[0] = 0  # Update to "Do Nothing" if constraint is violated

            if any(action != 0 for action in previous_actions_super) and inputs_cpu[0][1] >= 4:
                modified_action[1] = 0

            if any(action != 0 for action in previous_actions_sub) and inputs_cpu[0][2] >= 4:
                modified_action[2] = 0


            # uppdate the suggested action based on the correlation of components
            if modified_action[2] == 3:
                modified_action[0] = 3
                modified_action[1] = 3

            if modified_action[1] == 3:
                modified_action[0] = 3


            previous_actions_deck.append(modified_action[0])

            previous_actions_super.append(modified_action[1])

            previous_actions_sub.append(modified_action[2])

            return torch.tensor([np.asarray(np.ravel_multi_index(modified_action, (4, 4, 4)))], device=device, dtype=torch.long)


    else:
        # Exploration: Choose a random action
        random_action = env.action_space.sample()
        modified_action = np.asarray(np.unravel_index(random_action, (4, 4, 4)))
        inputs_cpu = inputs.to('cpu').numpy()


        # Update the previous_actions list for each component
        previous_actions_deck.append(modified_action[0])

        previous_actions_super.append(modified_action[1])

        previous_actions_sub.append(modified_action[2])

        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)



print('Trainig started')
policy_net = DQN(n_observations+n_params,n_actions).to(device)
target_net = DQN(n_observations+n_params,n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.Adam(policy_net.parameters(), lr=LR)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1250, gamma=0.95)
memory = PrioritizedReplayMemory(int(1e5))
target_model_update = int(40)

i_episode = 0;
test_on = 0




episode_reward = []

## Plot training curves
def plot_training(show_result=False):
    plt.figure(1)
    metric_temp = torch.tensor(episode_reward, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Episode cumulative cost')
    plt.plot(metric_temp.numpy())
    # Take 100 episode averages and plot them too
    if len(metric_temp) >= 50:
        means = metric_temp.unfold(0, 50, 1).mean(1).view(-1)
        #means = torch.cat((torch.zeros(49), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())



def optimize_model():
    if len(memory) < BATCH_SIZE:
        return

    # Sample a batch from the prioritized replay memory
    transitions, indices, weights = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                            batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                      if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    A_Deck_batch = torch.cat(batch.A_deck)
    L_D_batch = torch.cat(batch.L_D)
    ADT_batch = torch.cat(batch.ADT)
    r_truck_batch = torch.cat(batch.r_truck)
    input_batch = torch.hstack((state_batch, A_Deck_batch, L_D_batch, ADT_batch, r_truck_batch))

    state_action_values = policy_net(input_batch).gather(1, action_batch)
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        input_batch_target = torch.hstack((non_final_next_states, A_Deck_batch[non_final_mask],
                                          L_D_batch[non_final_mask], ADT_batch[non_final_mask],
                                          r_truck_batch[non_final_mask]))
        next_state_values[non_final_mask] = target_net(input_batch_target).max(1)[0]


    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    criterion = nn.MSELoss()
    # Compute Huber loss
    loss = criterion(state_action_values, torch.tensor(expected_state_action_values, dtype=torch.float32).unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Update priorities in the memory
    updated_priorities = torch.abs(expected_state_action_values - state_action_values.squeeze(1))
    updated_priorities = updated_priorities.cpu().detach().numpy()
    memory.update_priorities(indices, updated_priorities)


# Latin Hypercube Sampling of bridge random parameters
from scipy.stats import qmc
LHS_sampler = qmc.LatinHypercube(d=n_params)
LHS_samples = LHS_sampler.random(n=num_episodes)

# Convert the LHS sample into each marginal distributions
deck_area_LHS = 100 + (1200-100)*LHS_samples[:,0]
L_d_LHS = 0.5 + (20-0.5)*LHS_samples[:,1]
ADT_LHS = 500 + (20000-500)*LHS_samples[:,2]
r_Truck_LHS = 0.00 + (0.18-0.00)*LHS_samples[:,3]


for i_episode in range(num_episodes):
    scheduler.step()

    state = env.reset()

    deck_area = deck_area_LHS[i_episode]
    L_d = L_d_LHS[i_episode]
    ADT = ADT_LHS[i_episode]
    r_Truck = r_Truck_LHS[i_episode]


    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    deck_area = torch.tensor([deck_area], dtype=torch.float32, device=device).unsqueeze(0)
    L_d = torch.tensor([L_d], dtype=torch.float32, device=device).unsqueeze(0)
    ADT = torch.tensor([ADT], dtype=torch.float32, device=device).unsqueeze(0)
    r_Truck = torch.tensor([r_Truck], dtype=torch.float32, device=device).unsqueeze(0)

    episode_reward_temp = 0.0
    terminated = False

    previous_actions_deck = deque([], maxlen=5)
    previous_actions_super = deque([], maxlen=5)
    previous_actions_sub = deque([], maxlen=5)

    t = int(0)
    while not terminated:
        input_temp = torch.hstack((state, deck_area, L_d, ADT, r_Truck))
        input_temp = input_temp.to(device)  # Ensure data is on the correct device
        action = select_action(input_temp)
        observation, _,_,reward, terminated, _,damage= env.step(action.item(), deck_area.item(), L_d.item(), ADT.item(), r_Truck.item(), jacketing = steel_lacketing, jacketing_t = time_jacketing, extender = seat_extender, extender_t = time_extender, shearKey = shear_key, shearKey_t = time_shearKey, w1=w1)
        episode_reward_temp += reward * (GAMMA ** t)
        reward = torch.tensor([reward], device=device)
        done = terminated
        t += 1
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward, deck_area, L_d, ADT, r_Truck)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        if done:
            episode_reward.append(-episode_reward_temp)
            plot_training()
            break

    # Update the target model every target_model_update episodes
    if i_episode % target_model_update == 0:
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]
        target_net.load_state_dict(target_net_state_dict)






###############################################################################
############### Save the trained model (AI-agent) for later use ###############
###############################################################################
print('Trainig completed')

torch.save(policy_net.state_dict(), 'Trained_model.pth') # Save model
np.save('Trained_model',episode_reward) # save the training results

###############################################################################
###############################################################################

plot_training(show_result=True)
plt.ioff()
plt.show()



print("--- %s seconds ---" % (time.time() - start_time))

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>